In [1]:
!pip install pulp

In [2]:
from pulp import *
# Creates a list of all the supply nodes
supply_nodes = ["S1","S2","S3"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"S1": 11,
      "S2": 13,
      "S3": 19}

# Creates a list of all demand nodes
demand_nodes = ["D1","D2","D3" ,"D4"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"D1": 6,
      "D2": 10,
      "D3": 12,
      "D4":15}
# Creates a list of costs of each transportation path
costs = [# Demand
     #D1 D2 D3 D4
     [21,16,25,13], #S1
     [17,18,14,23], #S2  Supply
     [32,27,18,41]  #S3
     ]

costs = makeDict((supply_nodes, demand_nodes),costs)
print(costs)
# Creates the prob variable to contain the problem data
prob = LpProblem("Product Distribution Problem",LpMinimize)
# Creates a list of tuples containing all the possible routes for transport
Routes = [(s,d) for s in supply_nodes for d in demand_nodes]
# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = LpVariable.dicts("Route",(supply_nodes,demand_nodes),0,None,LpInteger)
# The objective function is added to prob first
prob += lpSum([route_vars[s][d]*costs[s][d] for (s,d) in Routes]), "Sum of Transporting Costs"
# The supply maximum constraints are added to prob for each supply node (warehouse)
for s in supply_nodes:
    prob += lpSum([route_vars[s][d] for d in demand_nodes]) <= supply[s], "Sum of Products out of supply %s"%s

# The demand minimum constraints are added to prob for each demand node (bar)
for d in demand_nodes:
    prob += lpSum([route_vars[s][d] for s in supply_nodes]) >= demand[d], "Sum of Products into demand %s"%d
prob.solve()


{'S1': {'D1': 21, 'D2': 16, 'D3': 25, 'D4': 13}, 'S2': {'D1': 17, 'D2': 18, 'D3': 14, 'D4': 23}, 'S3': {'D1': 32, 'D2': 27, 'D3': 18, 'D4': 41}}


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

In [3]:
print("Status:",LpStatus[prob.status] )

Status: Optimal


In [4]:
total=0
for v in prob.variables():
    if(v.varValue != None):
        if (v.varValue > 0):
            print(v.name, "=", v.varValue)
            total+=v.varValue


Route_S1_D4 = 11.0
Route_S2_D1 = 6.0
Route_S2_D2 = 3.0
Route_S2_D4 = 4.0
Route_S3_D2 = 7.0
Route_S3_D3 = 12.0


In [5]:
print("Optimal Cost is = ",(13*11)+(17*6)+(18*3)+(23*4)+(27*7)+(18*12))

Optimal Cost is =  796
